In [1]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install -U gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Урок 4. Тематическое моделирование. EM-алгоритм

Задача тематическое моделирование

продолжаем исследование датасета с твитами

Скачиваем датасет (источник): положительные, отрицательные.

или можно через ноутбук

!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

как альтернатива можно скачать данные из Роспотребнадзора
https://zpp.rospotrebnadzor.ru/Forum/Appeals
для этого берём ноутбук parse_rospotrebnadzor.ipynb
устанавливаем количество скачанных страниц больше не 50-сят хотябы 500 и для анализа берём только вопросы так как ответы есть не всегда

что надо сделать
1. объединить в одну выборку (это только для твитов), для роспотребнадзора сформировать датасет из вопросов

In [6]:
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

--2022-06-11 07:52:27--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6022:18::a27d:4212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2022-06-11 07:52:27--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd82863b731d3fe8aa4c170adc1.dl.dropboxusercontent.com/cd/0/inline/Bm-FIFq-2ulUWi-YaU5jW_Vip2e7tmNPkKJ2yAJNTiCWWRc4xzB1qHI3yJv4uce76MFbkg-0tyLtrd1aXjNQ8zMp_dcKkOHIhd9WuKLxu1lKTseo_90jKZszeJJyCKDCRwuwkFDw0CSGWw37VDdiid8k1uLhPha5Orz06ekil8XIdg/file# [following]
--2022-06-11 07:52:28--  https://ucd82863b731d3fe8aa4c170adc1.dl.dropboxusercontent.com/cd/0/inline/Bm-FIFq-2ulUWi-YaU5jW_Vip2e7tmNPkKJ2yAJNTiCWWRc4xzB1qHI3yJv4uce76MF

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [9]:
import string
import pymorphy2
from stop_words import get_stop_words
import annoy
import re
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from gensim.models import *
from gensim import corpora
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

from sklearn.metrics import *
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


In [10]:
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])

negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])

df = positive.append(negative)

#df = df.reset_index().drop('index', axis=1)

In [11]:
df.head()

,text
0,"@first_timee хоть я и школота, но поверь, у на..."
1,"Да, все-таки он немного похож на него. Но мой ..."
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...
3,"RT @digger2912: ""Кто то в углу сидит и погибае..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...


2. провести исследование и выявить тематики о которых говорят в твитах (для твитов), а для роспотребнадзора так же выявить тематики о которых люди пишут проанализировать

In [12]:
#df = df.sample(10000)

In [13]:
morph = pymorphy2.MorphAnalyzer()

/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.g

In [14]:
stopwords = get_stop_words("ru") + ['http']

In [15]:
words_regex = re.compile('\w+')

def find_words(text, regex = words_regex):
    tokens =  regex.findall(text.lower())
    return [w for w in tokens if w.isalpha() and len(w) >= 3]

def lemmatize(words, lemmer = morph, stopwords = stopwords):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words]
    return [w for w in lemmas if not w in stopwords 
            and w.isalpha()]

def preprocess(text):
    return (lemmatize(find_words(text)))

<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<ipython-input-15-78e5866c10d0>:1: DeprecationWarning: invalid escape sequence \w
  words_regex = re.compile('\w+')


In [16]:
preprocessed_text = list(map(preprocess, df['text']))

In [17]:
df['prepared_text'] = preprocessed_text

In [18]:
dictionary = corpora.Dictionary(df['prepared_text'])

dictionary.filter_extremes(no_below = 10, no_above = 0.90, keep_n=None)  
dictionary.save('lenta.dict')

In [19]:
corpus = [dictionary.doc2bow(text) for text in df['prepared_text']]
corpora.MmCorpus.serialize('lenta.model', corpus) 

In [20]:
lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=10, chunksize=50, update_every=1, passes=2)

3. сделать визуализацию кластеров тематик

In [21]:
lda.show_topics(num_topics=10, num_words=10, formatted=True)

[(0,
  '0.045*"знать" + 0.033*"делать" + 0.033*"любить" + 0.031*"большой" + 0.030*"спать" + 0.027*"хороший" + 0.017*"сделать" + 0.016*"бля" + 0.014*"посмотреть" + 0.013*"нибыть"'),
 (1,
  '0.060*"пойти" + 0.036*"работа" + 0.035*"скоро" + 0.035*"плохой" + 0.019*"температура" + 0.018*"уехать" + 0.017*"пол" + 0.016*"зима" + 0.016*"целый" + 0.015*"учиться"'),
 (2,
  '0.063*"завтра" + 0.053*"блин" + 0.040*"идти" + 0.036*"школа" + 0.036*"писать" + 0.027*"хотеться" + 0.024*"урок" + 0.021*"последний" + 0.019*"любимый" + 0.018*"улица"'),
 (3,
  '0.038*"самый" + 0.029*"утро" + 0.028*"понимать" + 0.027*"неделя" + 0.020*"пиздец" + 0.017*"казаться" + 0.017*"рука" + 0.016*"устать" + 0.015*"твит" + 0.015*"обидно"'),
 (4,
  '0.071*"свой" + 0.053*"скучать" + 0.038*"стать" + 0.022*"amp" + 0.021*"вечер" + 0.020*"сильно" + 0.019*"ходить" + 0.019*"парень" + 0.019*"давать" + 0.016*"играть"'),
 (5,
  '0.039*"понять" + 0.030*"понедельник" + 0.030*"учёба" + 0.023*"уснуть" + 0.020*"поход" + 0.019*"помочь" + 0.0

In [22]:
vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


4. проинтерпритировать получившиеся тематики

Тема 1: Выбор

Тема 2: Работа

Тема 3: Школа

Тема 4: Усталось 

Тема 5: Романтика

Тема 6: Учеба

Тема 7: Болезнь

Тема 8: Отдых

Тема 9: Страх

Тема 10: Ожинание